#### Representing Progarm by Tree

In [1]:
from random import random,randint,choice
from copy import deepcopy
from math import log

class fwrapper:
    def __init__(self,function,childcount,name):
        self.function=function
        self.childcount=childcount
        self.name=name
class node:
    def __init__(self,fw,children):
        self.function=fw.function
        self.name=fw.name
        self.children=children
    def evaluate(self,inp):
        results=[n.evaluate(inp) for n in self.children]
        return self.function(results)
    def display(self,indent=0):
        print (' '*indent, self.name)
        for c in self.children:
            c.display(indent+1)
    
class paramnode:
    def __init__(self,idx):
        self.idx=idx
    def evaluate(self,inp):
        return inp[self.idx]
    def display(self,indent=0):
         print('%sp%d' % (' '*indent,self.idx))
    
class constnode:
    def __init__(self,v):
        self.v=v
    def evaluate(self,inp):
        return self.v
    def display(self,indent=0):
         print('%s%d' % (' '*indent,self.v))

#### Define Functions

In [2]:
addw=fwrapper(lambda l:l[0]+l[1],2,'add')
subw=fwrapper(lambda l:l[0]-l[1],2,'subtract')
mulw=fwrapper(lambda l:l[0]*l[1],2,'multiply')
def iffunc(l):
    if l[0]>0: return l[1]
    else: return l[2]
ifw=fwrapper(iffunc,3,'if')

def isgreater(l):
    if l[0]>l[1]: return 1
    else: return 0
gtw=fwrapper(isgreater,2,'isgreater')

flist=[addw,mulw,ifw,gtw,subw]

#### Example Program
Below is a tree representation of a simple program, involving one if-statement.

In [3]:
def example(p0, p1):
    if p0 > 3:
        return p1 + 5
    else:
        return p1 - 2

In [4]:
def exampletree(): 
    return node(ifw,[node(gtw,[paramnode(0),constnode(3)]),
                     node(addw,[paramnode(1),constnode(5)]),
                       node(subw,[paramnode(1),constnode(2)])])
tree = exampletree()
tree.evaluate([2,3])

1

In [5]:
tree.display()

 if
  isgreater
  p0
  3
  add
  p1
  5
  subtract
  p1
  2


#### Initial Population

In [6]:
def makerandomtree(pc,maxdepth=4,fpr=0.5,ppr=0.6): 
    if random()<fpr and maxdepth>0:
        f=choice(flist)
        children=[makerandomtree(pc,maxdepth-1,fpr,ppr)
               for i in range(f.childcount)]
        return node(f,children)
    elif random()<ppr:
        return paramnode(randint(0,pc-1))
    else:
        return constnode(randint(0,10))

In [7]:
random1=makerandomtree(2, 0.9)
random1.evaluate([7,1])

0

In [8]:
random1.display()

 isgreater
 p1
 4


In [9]:
random2=makerandomtree(2)
random2.evaluate([5,3])

2

In [10]:
random2.display()

2


#### Testing 

In [11]:
def hiddenfunction(x,y):
    return x*y+5

def buildhiddenset(): 
    rows=[]
    for i in range(200):
        x=randint(0,40)
        y=randint(0,40)
        rows.append([x,y,hiddenfunction(x,y)])
    return rows

hiddenset=buildhiddenset()

In [12]:
# measure error using mean absolute error
def scorefunction(tree,s):
    dif=0
    for data in s:
        v=tree.evaluate([data[0],data[1]])
        dif+=abs(v-data[2])
    return dif

In [13]:
scorefunction(random1,hiddenset)

78810

#### Mutation

In [14]:
def mutate(t,pc,probchange=0.1): 
    if random()<probchange:
        return makerandomtree(pc)
    else:
        result=deepcopy(t)
        if isinstance(t,node):
            result.children=[mutate(c,pc,probchange) for c in t.children]
        return result

In [15]:
random2.display()

2


In [16]:
muttree=mutate(random2,2)
muttree.display()

2


#### Crossover

In [17]:
def crossover(t1,t2,probswap=0.7,top=1): 
    if random()<probswap and not top:
        return deepcopy(t2)
    else:
        result=deepcopy(t1)
        if isinstance(t1,node) and isinstance(t2,node):
            result.children=[crossover(c,choice(t2.children),probswap,0)
                            for c in t1.children]
        return result

In [18]:
random1=makerandomtree(2)
random1.display()

 add
 p0
  isgreater
  6
  5


In [19]:
random2=makerandomtree(2)
random2.display()

 isgreater
  if
  p0
   if
   p1
    add
    5
    p1
   p0
  p1
 p1


In [20]:
cross=crossover(random1,random2)
cross.display()

 add
  if
  p0
   if
   p1
    add
    5
    p1
   p0
  p1
  isgreater
  6
  5


#### Environment

In [21]:
def evolve(pc,popsize,rankfunction,maxgen=500,
                mutationrate=0.1,breedingrate=0.4,pexp=0.7,pnew=0.05):
    # Returns a random number, tending towards lower numbers. 
    # The lower pexp # is, more lower numbers you will get
    def selectindex():
        return int(log(random())/log(pexp))
    # Create a random initial population
    population=[makerandomtree(pc) for i in range(popsize)]
    for i in range(maxgen):
        scores=rankfunction(population)
        print("Generation %i, cost: %i"%(i, scores[0][0]))
        if scores[0][0]==0: break
        # The two best always make it
        newpop=[scores[0][1],scores[1][1]]
        
        # Build the next generation
        while len(newpop)<popsize:
            if random()>pnew: 
                newpop.append(mutate(crossover(scores[selectindex( )][1], 
                                               scores[selectindex( )][1],
                                               probswap=breedingrate),
                                     pc,probchange=mutationrate))
            else:
                # Add a random node to mix things up
                newpop.append(makerandomtree(pc))
        population=newpop 
    return scores[0][1]

In [22]:
def getrankfunction(dataset):
    def rankfunction(population):
        scores=[(scorefunction(t,dataset),t) for t in population] 
        scores.sort(key = lambda i : i[0])
        return scores
    return rankfunction

#### Evolution / Training

In [23]:
rf=getrankfunction(buildhiddenset())

In [25]:
best = evolve(2,500,rf,mutationrate=0.2,breedingrate=0.1,pexp=0.7,pnew=0.2)

Generation 0, cost: 3793
Generation 1, cost: 98
Generation 2, cost: 0


In [26]:
best.display( ) 

 add
  isgreater
   multiply
   0
    multiply
    1
     if
     2
     8
     p0
   isgreater
   p1
    add
     multiply
     p1
     p1
    0
  add
   add
   5
    multiply
    p1
    p0
   isgreater
    subtract
    p1
    p1
    isgreater
    p1
     isgreater
      multiply
      p1
      8
     6
